In [1]:
import numpy as np
import tensorcircuit as tc
import tensorflow as tf
K = tc.set_backend("tensorflow")
ctype, rtype = tc.set_dtype("complex128")

Please first ``pip install -U qiskit`` to enable related functionality in translation module


In [17]:
ctype

'complex128'

In [18]:
rtype

'float64'

In [2]:
def rx0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._x_matrix, K.eye(2)
    )


def rx1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._x_matrix
    )


def ry0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._y_matrix, K.eye(2)
    )


def ry1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._y_matrix
    )


def rz0(theta):
    return K.kron(
        K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._z_matrix, K.eye(2)
    )


def rz1(theta):
    return K.kron(
        K.eye(2), K.cos(theta) * K.eye(2) + 1.0j * K.sin(theta) * tc.gates._z_matrix
    )


def cnot01():
    return K.cast(K.convert_to_tensor(tc.gates._cnot_matrix), ctype)


def cnot10():
    return K.cast(
        K.convert_to_tensor(
            np.array([[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
        ),
        ctype,
    )


ops_repr = ["rx0", "rx1", "ry0", "ry1", "rz0", "rz1", "cnot01", "cnot10"]

In [3]:
n, p, ch = 2, 3, 8
# 量子比特数、层数、操作池大小

target = tc.array_to_tensor(np.array([1, 0, 0, 1.0]) / np.sqrt(2.0))
# 目标波函数，我们这里使用 GHZ2 状态作为目标函数


def ansatz(params, structures):
    c = tc.Circuit(n)
    params = K.cast(params, ctype)
    structures = K.cast(structures, ctype)
    for i in range(p):
        c.any(
            0,
            1,
            unitary=structures[i, 0] * rx0(params[i, 0])
            + structures[i, 1] * rx1(params[i, 1])
            + structures[i, 2] * ry0(params[i, 2])
            + structures[i, 3] * ry1(params[i, 3])
            + structures[i, 4] * rz0(params[i, 4])
            + structures[i, 5] * rz1(params[i, 5])
            + structures[i, 6] * cnot01()
            + structures[i, 7] * cnot10(),
        )
    s = c.state()
    loss = K.sum(K.abs(target - s))
    return loss


vag1 = K.jit(K.vvag(ansatz, argnums=0, vectorized_argnums=1))

In [4]:
def sampling_from_structure(structures, batch=1):
    prob = K.softmax(K.real(structures), axis=-1)
    return np.array([np.random.choice(ch, p=K.numpy(prob[i])) for i in range(p)])


@K.jit
def best_from_structure(structures):
    return K.argmax(structures, axis=-1)


@K.jit
def nmf_gradient(structures, oh):
    """
    根据朴素平均场概率模型计算蒙特卡洛梯度
    """
    choice = K.argmax(oh, axis=-1)
    prob = K.softmax(K.real(structures), axis=-1)
    indices = K.transpose(K.stack([K.cast(tf.range(p), "int64"), choice]))
    prob = tf.gather_nd(prob, indices)
    prob = K.reshape(prob, [-1, 1])
    prob = K.tile(prob, [1, ch])

    return tf.tensor_scatter_nd_add(
        tf.cast(-prob, dtype=ctype),
        indices,
        tf.ones([p], dtype=ctype),
    )


nmf_gradient_vmap = K.vmap(nmf_gradient, vectorized_argnums=1)

In [5]:
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
stp = K.implicit_randn(stddev=0.02, shape=[p, 8], dtype=rtype)

In [6]:
stp

<tf.Tensor: shape=(3, 8), dtype=float64, numpy=
array([[ 0.03549599,  0.00259103,  0.02050921,  0.04924461, -0.00708516,
         0.03031192,  0.00734741,  0.00289678],
       [-0.01013225, -0.00139695,  0.01322092,  0.00611694,  0.01441615,
         0.01181014,  0.01603067,  0.01049271],
       [-0.00903826, -0.025523  , -0.0189485 ,  0.02152791,  0.00156384,
         0.01501239, -0.00063014,  0.03422753]])>

In [5]:
verbose = False
epochs = 400
batch = 256
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.06, 100, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.12))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
stp = K.implicit_randn(stddev=0.02, shape=[p, 8], dtype=rtype)
avcost1 = 0
for epoch in range(epochs):  # 更新结构参数的迭代
    avcost2 = avcost1
    costl = []
    batched_stuctures = K.onehot(
        np.stack([sampling_from_structure(stp) for _ in range(batch)]), num=8
    )
    infd, gnnp = vag1(nnp, batched_stuctures)
    gs = nmf_gradient_vmap(stp, batched_stuctures)  # \nabla lnp
    gstp = [K.cast((infd[i] - avcost2), ctype) * gs[i] for i in range(infd.shape[0])]
    gstp = K.real(K.sum(gstp, axis=0) / infd.shape[0])
    avcost1 = K.sum(infd) / infd.shape[0]
    nnp = network_opt.update(gnnp, nnp)
    stp = structure_opt.update(gstp, stp)

    if epoch % 40 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched average loss: ",
            np.mean(avcost1),
        )

        if verbose:
            print(
                "strcuture parameter: \n",
                stp.numpy(),
                "\n network parameter: \n",
                nnp.numpy(),
            )

        cand_preset = best_from_structure(stp)
        print("best candidates so far:", [ops_repr[i] for i in cand_preset])
        print(
            "corresponding weights for each gate:",
            [K.numpy(nnp[j, i]) if i < 6 else 0.0 for j, i in enumerate(cand_preset)],
        )

----------epoch 0-----------
batched average loss:  1.4350284930905612
best candidates so far: ['ry1', 'rz0', 'rx0']
corresponding weights for each gate: [0.03687752986503738, -0.047695993774942755, 0.04552400264153701]
----------epoch 40-----------
batched average loss:  1.013661786785497
best candidates so far: ['rz1', 'rz0', 'rz0']
corresponding weights for each gate: [-0.00338954721777787, 0.003513532294152719, 0.0031886893969231165]
----------epoch 80-----------
batched average loss:  1.0001894815595078
best candidates so far: ['rz1', 'rz0', 'rz0']
corresponding weights for each gate: [0.00045108170240731665, -0.0013252198169867763, -0.0003939969594297074]
----------epoch 120-----------
batched average loss:  1.0001383514980922
best candidates so far: ['rz1', 'rz0', 'rz0']
corresponding weights for each gate: [8.119327330189437e-07, 9.39591774612522e-05, 3.9127445710219864e-05]
----------epoch 160-----------
batched average loss:  1.000065754484419
best candidates so far: ['rz1', 

In [6]:
def ansatz2(params, structures):
    c = tc.Circuit(n)
    params = K.cast(params, ctype)
    structures = K.softmax(structures, axis=-1)
    structures = K.cast(structures, ctype)
    for i in range(p):
        c.any(
            0,
            1,
            unitary=structures[i, 0] * rx0(params[i, 0])
            + structures[i, 1] * rx1(params[i, 1])
            + structures[i, 2] * ry0(params[i, 2])
            + structures[i, 3] * ry1(params[i, 3])
            + structures[i, 4] * rz0(params[i, 4])
            + structures[i, 5] * rz1(params[i, 5])
            + structures[i, 6] * cnot01()
            + structures[i, 7] * cnot10(),
        )
    s = c.state()
    s /= K.norm(s)
    loss = K.sum(K.abs(target - s))
    return loss


vag2 = K.jit(K.value_and_grad(ansatz2, argnums=(0, 1)))

In [7]:
verbose = True
epochs = 700
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.05, 200, 0.5)
structure_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(0.04))
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(lr))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
stp = K.implicit_randn(stddev=0.02, shape=[p, 8], dtype=rtype)
for epoch in range(epochs):
    infd, (gnnp, gstp) = vag2(nnp, stp)

    nnp = network_opt.update(gnnp, nnp)
    stp = structure_opt.update(gstp, stp)
    if epoch % 70 == 0 or epoch == epochs - 1:
        print("----------epoch %s-----------" % epoch)
        print(
            "batched average loss: ",
            np.mean(infd),
        )
        if verbose:
            print(
                "strcuture parameter: \n",
                stp.numpy(),
                "\n network parameter: \n",
                nnp.numpy(),
            )

        cand_preset = best_from_structure(stp)
        print("best candidates so far:", [ops_repr[i] for i in cand_preset])
        print(
            "corresponding weights for each gate:",
            [K.numpy(nnp[j, i]) if i < 6 else 0.0 for j, i in enumerate(cand_preset)],
        )

----------epoch 0-----------
batched average loss:  1.3028580976115745
strcuture parameter: 
 [[ 0.02642551  0.04879337  0.03993834  0.05393528  0.01007346  0.06594783
   0.02178556 -0.065861  ]
 [ 0.03243891  0.05765078  0.04342524  0.03790864  0.03944079  0.01919895
   0.06946815 -0.0165585 ]
 [ 0.01991238  0.02745579  0.05845886  0.04868942  0.09504317  0.07270755
   0.04895157 -0.0589975 ]] 
 network parameter: 
 [[-0.04931527  0.01637837 -0.02725512  0.05935174  0.02434643  0.02549461]
 [-0.03793334  0.04774108 -0.06644895  0.07115726  0.04545609  0.01796591]
 [-0.03906845  0.05768943 -0.05711509  0.0408206   0.07229365  0.04936205]]
best candidates so far: ['rz1', 'cnot01', 'rz0']
corresponding weights for each gate: [0.025494614256898068, 0.0, 0.07229365328164147]
----------epoch 70-----------
batched average loss:  0.6702069440721292
strcuture parameter: 
 [[-0.24716255  0.08023233  1.95626993  0.54401311  0.1323524   0.18822557
   0.16744054 -0.82487636]
 [-0.066985    0.32714

In [8]:
chosen_structure = K.onehot(np.array([2, 4, 6]), num=8)
chosen_structure = K.reshape(chosen_structure, [1, p, ch])
chosen_structure

<tf.Tensor: shape=(1, 3, 8), dtype=float32, numpy=
array([[[0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.]]], dtype=float32)>

In [12]:
network_opt = tc.backend.optimizer(tf.keras.optimizers.Adam(1e-3))
nnp = K.implicit_randn(stddev=0.02, shape=[p, 6], dtype=rtype)
verbose = True
epochs = 1000
for epoch in range(epochs):
    infd, gnnp = vag1(nnp, chosen_structure)
    nnp = network_opt.update(gnnp, nnp)
    if epoch % 60 == 0 or epoch == epochs - 1:
        print(epoch, "loss: ", K.numpy(infd[0]))

0 loss:  0.9644019550577962
60 loss:  0.8989188689211314
120 loss:  0.8298012603554974
180 loss:  0.7560331634927562
240 loss:  0.6779941679539283
300 loss:  0.5961486934180202
360 loss:  0.5110284342535933
420 loss:  0.4232176451899497
480 loss:  0.33333753027426183
540 loss:  0.24203042848994694
600 loss:  0.14994446358149527
660 loss:  0.05771922041773825
720 loss:  0.001956006605030141
780 loss:  0.00031044641307371524
840 loss:  0.0002432198430751021
900 loss:  0.00021240388204815484
960 loss:  0.0001003711168773016
999 loss:  0.00011424485522940562
